<a href="https://colab.research.google.com/github/JellyJoa/DataAnalysis/blob/master/Dacon/BlueHouse/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#%%
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tqdm import tqdm

In [9]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
train = pd.read_csv('/content/drive/MyDrive/DL/train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/drive/MyDrive/DL/test.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('/content/drive/MyDrive/DL/sample_submission.csv', encoding = 'utf-8')

In [11]:
train = train.dropna(how = 'any')
train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']
#%%

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
okt = Okt()

In [13]:
X_train = []
for sentence,i in zip(train['data'],tqdm(range(len(train['data'])))) :
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_train.append(temp_X)

100%|█████████▉| 39991/39992 [30:44<00:00, 21.69it/s]


In [14]:
X_test = []
for sentence in test['data']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_test.append(temp_X)

In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [16]:
vocab_size = 30000
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [17]:
max_len = 500
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [18]:
y_train = to_categorical(train['category'])

In [19]:
model = Sequential()
model.add(Embedding(vocab_size, 120))
model.add(LSTM(120))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=15)

Epoch 1/15
313/313 [==============================] - 19s 36ms/step - loss: 0.5305 - acc: 0.7893
Epoch 2/15
313/313 [==============================] - 11s 36ms/step - loss: 0.3119 - acc: 0.8899
Epoch 3/15
313/313 [==============================] - 11s 35ms/step - loss: 0.2279 - acc: 0.9203
Epoch 4/15
313/313 [==============================] - 11s 36ms/step - loss: 0.2130 - acc: 0.9257
Epoch 5/15
313/313 [==============================] - 11s 35ms/step - loss: 0.1616 - acc: 0.9415
Epoch 6/15
313/313 [==============================] - 11s 36ms/step - loss: 0.1365 - acc: 0.9511
Epoch 7/15
313/313 [==============================] - 11s 36ms/step - loss: 0.1441 - acc: 0.9473
Epoch 8/15
313/313 [==============================] - 11s 36ms/step - loss: 0.1156 - acc: 0.9579
Epoch 9/15
313/313 [==============================] - 11s 37ms/step - loss: 0.1445 - acc: 0.9497
Epoch 10/15
313/313 [==============================] - 11s 37ms/step - loss: 0.1117 - acc: 0.9609
Epoch 11/15
313/313 [========

In [22]:
y_pred = model.predict(X_test)
# predicted = y_prob.argmax(axis=-1)
sample_submission['category'] = y_pred
sample_submission.to_csv('/content/drive/MyDrive/DL/submission.csv', encoding='utf-8', index = False)

In [24]:
aa = pd.read_csv('/content/drive/MyDrive/DL/submission.csv')
print(aa)

      index  category
0         0  0.625299
1         1  0.000437
2         2  0.000005
3         3  0.646807
4         4  0.001793
...     ...       ...
4995   4995  0.999617
4996   4996  0.000028
4997   4997  0.000043
4998   4998  0.999692
4999   4999  0.918252

[5000 rows x 2 columns]
